In [1]:
import pandas as pd
import numpy as np
import os 
from bs4 import BeautifulSoup as bs
import requests

In [2]:
def upload_data(df, file_path):
    df.to_csv(file_path, index=False)

In [3]:
## Atleti History

In [4]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_Atl%C3%A9tico_Madrid_seasons'

In [5]:
def handle_goals (goal_series, repair_series):
    for i in range(0, len(goal_series), 1):
        if goal_series[i]==None:
            goal_series[i] = repair_series[i]
    return goal_series        

In [6]:
def handle_players(player_series):
    for i in range(len(player_series)):
        if player_series[i] is not None:
            if len(player_series[i]) <= 2:
                player_series[i] = player_series[i-1]
    return player_series

In [7]:
def extract_atleti_data_from_wikipedia(page_url):

    #sending request to access the webpage
    reponse = requests.get(page_url)
    soup = bs(reponse.text,"html.parser") 

    atleti_statistics = soup.find('table',{'class':"wikitable plainrowheaders"})

    #extracting row headers
    row_headers = []
    for x in atleti_statistics.find_all('tr')[0:]:
        for y in x.find_all('th'):
            item = y.text.replace('\n', '')
            row_headers.append(item)  
            
    #selecteing the needed headers
    headers = row_headers[:24]

    #removing unnecessary headers records
    headers.remove('Other competitions')
    headers.remove('League')

    #headers rearranging
    set_1 = headers[5:14]
    set_2 = headers[1:4]
    set_3 = headers[14:]

    table_headers =  set_1 + set_2 + set_3

    #extracting season feature 
    seasons = row_headers[24:]

    #extracting table records
    tableValues = []
    for x in atleti_statistics.find_all('tr')[1:]:
        td_tags = x.find_all('td')
        td_val = [y.text.replace('\n', '') for y in td_tags]
        tableValues.append(td_val)

    table_values = tableValues[1:]

    #Forming data frame that store atleti statistics 
    df = pd.DataFrame(table_values,columns=table_headers)

    #adding season feature to the dataframe
    df['season'] = pd.Series(seasons)

    #Handling Data inconsistency
    df['Goals'] = handle_goals(df['Goals'],df['Player(s)'])
    df['Player(s)'] = handle_players(df['Player(s)'])

    #Rmoving null values
    df = df.dropna()
    atelti_seasons_df = df

    ##Casting the features that should be in numerical form from categorical form into numerical form 
    series_that_should_converted_to_int = list(atelti_seasons_df.columns[1:8])
    series_that_should_converted_to_int.append('Goals')

    for i in series_that_should_converted_to_int:
        atelti_seasons_df[i] = atelti_seasons_df[i].astype(int)

    return atelti_seasons_df
    

In [8]:
atleti_history = extract_atleti_data_from_wikipedia(wikipedia_url)

In [9]:
atleti_history

,Division,Pld,W,D,L,GF,GA,Pts,Pos,Cup[1],...,LC,CL[3],UC[4],CW,IC,SC[5],CWC[6],Player(s),Goals,season
0,La Liga,18,8,2,8,43,41,18,6th,QF,...,,,,,,,,Cosme Vázquez,24,1928–29
1,La Liga,18,5,2,11,32,50,12,10th,R32,...,,,,,,,,Luis Marín Sabater,24,1929–30
2,Segunda División,18,11,1,6,47,31,23,3rd,R32,...,,,,,,,,Santiago Losada Amor [gl],15,1930–31
3,Segunda División,16,8,2,6,38,34,18,4th,R16,...,,,,,,,,Santiago Buiría,15,1931–32
4,Segunda División,18,12,3,3,58,26,27,2nd,R16,...,,,,,,,,Manuel Guijarro,17,1932–33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,La Liga,38,18,16,4,51,27,70,3rd,R32,...,,QF,,,,,,Álvaro Morata,16,2019–20
90,La Liga,38,26,8,4,67,25,86,W,2R,...,,R16,,,,,,Luis Suárez,21,2020–21
91,La Liga,38,21,8,9,65,43,71,3rd,R16,...,,QF,,,,,,Ángel Correa Luis Suárez,13,2021–22
92,La Liga,38,23,8,7,70,33,77,3rd,QF,...,,GS,,,,,,Antoine Griezmann,16,2022–23


In [10]:
upload_data(atleti_history, r'processed data/atleti_history_wikipedia.csv')

In [11]:
## UEFA Champions League  

In [12]:
def extract_transorm_uefa_data(file_path) :
    
    ## file path reading
    with open(file_path,'r') as text_file : 
        content = text_file.read()
        
    ## seprating data     
    separated_content = content.split('\n')

    ## Removing ['P', 'W', 'D', 'L'] 
    filtered_content = [item for item in separated_content if item not in ['P', 'W', 'D', 'L']]

    ## seperate ['P', 'W', 'D', 'L'] numbers from stage text 
    for i in range (1,len(filtered_content),2):
        filtered_content[i] = filtered_content[i].split(' ')

    ## dividing  filtered_content into season_list, session_stat to facilitate processing 
    season_list = []
    season_stat = []
    
    for i in range (0, len(filtered_content),1):
        if i%2==0 or i==0 :
            season_list.append(filtered_content[i])
        else :
            season_stat.append(filtered_content[i])

    ## Extracting Total played matches, Wins, Draws, Losses 
    total_played_matches = [] 
    wins_list = []
    draw_list = []
    loses_list = []

    for i in range (0, len(season_stat),1):
        total_played_matches.append(season_stat[i][-4])
        wins_list.append(season_stat[i][-3])
        draw_list.append(season_stat[i][-2])
        loses_list.append(season_stat[i][-1])

    ## Extracting Stages Column
    trimmed_season_stat = [sublist[:-4] for sublist in season_stat]
    stage = [' '.join(sublist) for sublist in trimmed_season_stat]

    ## Forming dataframe
    data_dic = { 'season': season_list,
             'stage' : stage, 
             'played': total_played_matches, 
             'win' : wins_list,
             'draw': draw_list, 
             'lose': loses_list
              }
    df = pd.DataFrame(data_dic)
    return df

In [13]:
atleti_in_ucl_df = extract_transorm_uefa_data('text data ucl/atleti_history_in_ucl.txt')

In [14]:
## UCL 

In [15]:
atleti_in_ucl_df

,season,stage,played,win,draw,lose
0,2023/24,Quarter-finals,10,5,3,2
1,2022/23,Group stage,6,1,2,3
2,2021/22,Quarter-finals,10,3,3,4
3,2020/21,Round of 16,8,2,3,3
4,2019/20,Quarter-finals,9,4,1,4
5,2018/19,Round of 16,8,5,1,2
6,2017/18,Group stage,6,1,4,1
7,2016/17,Semi-finals,12,8,2,2
8,2015/16,Final,13,6,4,3
9,2014/15,Quarter-finals,10,4,3,3


In [16]:
## Europe League 

In [17]:
atleti_in_europe_league_df = extract_transorm_uefa_data('text data europe league/atleti_history_europe_league.txt')

In [18]:
atleti_in_europe_league_df

,season,stage,played,win,draw,lose
0,2017/18,Final,9,7,1,1
1,2012/13,Round of 32,8,5,0,3
2,2011/12,Final,19,17,1,1
3,2010/11,Group stage,6,2,2,2
4,2009/10,Final,9,3,5,1
5,2007/08,Round of 32,10,7,2,1
6,1999/00,Fourth round,8,5,1,2
7,1998/99,Semi-finals,10,7,0,3
8,1997/98,Semi-finals,10,5,3,2
9,1993/94,Second round,4,2,0,2


#### Uefa Goals States piplines (uefa champions league, uefa europe league) --> Group stages matches & play off matches

In [19]:
def divide_list_by_distance(separated_content, distance):
    list_of_list = []
    
    for i in range(0, len(separated_content), distance):
        if i + distance > len(separated_content):  
            break
        sliced_list = separated_content[i:i + distance]  # Slice the list inside the loop
        list_of_list.append(sliced_list)
    
    return list_of_list

In [20]:
def extract_atleti_stats_from_UEFA_group_stage(file_path):

    #Reading file path
    with open(file_path, 'r') as text_file:
        content = text_file.read()

    #converting text into list
    separated_content = content.split('\n')

    #Slicing list to be 6 lists inside one list to deal with each list as one record
    group_stage_list = divide_list_by_distance (separated_content,6)

    #the need lists to store data in their right postion  
    season_list = []
    total_goals_scored = []
    total_goals_conceded = []
    diff_between_score_conced = [] 
    total_points =[]
    postion_in_group_stage = []

    # inserting data to the right list
    for i in range (0, len(group_stage_list),1):    
        season_list.append(group_stage_list[i][-6])
        total_goals_scored.append(group_stage_list[i][-5])
        total_goals_conceded.append(group_stage_list[i][-4])
        diff_between_score_conced.append(group_stage_list[i][-3])
        total_points.append(group_stage_list[i][-2])
        postion_in_group_stage.append(group_stage_list[i][-1])

    # store data in dictionary to form dataframe
    dict = {'season' : season_list, 
        'scored goals' : total_goals_scored,
        'conceded goals' : total_goals_conceded, 
        'diff' : diff_between_score_conced,
        'total points' : total_points,
        'postion in group':postion_in_group_stage
     }

    df = pd.DataFrame(dict)
    return df

In [21]:
#Group Stage uefa champions league
atleti_group_stage_simone_ucl_df = extract_atleti_stats_from_UEFA_group_stage('text data ucl/simone_in_group_stages_ucl.txt')

In [22]:
atleti_group_stage_simone_ucl_df

,season,scored goals,conceded goals,diff,total points,postion in group
0,2013/2014,15,3,12,16,1
1,2014/2015,14,3,11,13,1
2,2015/2016,11,3,8,13,1
3,2016/2017,7,2,5,15,1
4,2017/2018,5,4,1,7,3
5,2018/2019,9,6,3,13,2
6,2019/2020,8,5,3,10,2
7,2020/2021,7,8,-1,9,2
8,2021/2022,7,8,-1,7,2
9,2022/2023,5,9,-4,5,4


In [23]:
#Group Stage uefa europe league

In [24]:
atleti_group_stage_europe_league = extract_atleti_stats_from_UEFA_group_stage('text data europe league/group_stage_ul_simone.txt')

In [25]:
atleti_group_stage_europe_league

,season,scored goals,conceded goals,diff,total points,postion in group
0,2011/2012,11,4,7,13,1


In [26]:
upload_data(atleti_group_stage_europe_league, 'processed data/atleti_group_stage_europe_league.csv')

## Atleti play off matches in ucl with deigo simone

In [27]:
def extract_atleti_stats_from_UEFA_Play_off_rounds(file_path):

    #Reading file path
    with open(file_path, 'r') as text_file:
        content = text_file.read()

    #converting text into list
    separated_content = content.split('\n')

    #Slicing list to be 6 lists inside one list to deal with each list as one record
    Play_off_rounds_list = divide_list_by_distance (separated_content,5)

    #the need lists to store data in their right postion  
    season_list = []
    atleti_first_match = []
    competitor_first_match = []
    atleti_second_match = [] 
    competitor_second_match =[]


    # inserting data to the right list
    for i in range (0, len(Play_off_rounds_list),1):    
        
        season_list.append(Play_off_rounds_list[i][-5])
        atleti_first_match.append(Play_off_rounds_list[i][-4])
        competitor_first_match.append(Play_off_rounds_list[i][-3])
        atleti_second_match.append(Play_off_rounds_list[i][-2])
        competitor_second_match.append(Play_off_rounds_list[i][-1])

    # store data in dictionary to form dataframe
    dict = {'season' : season_list, 
        'atleti first match' : atleti_first_match,
        'competitor first match' : competitor_first_match, 
        'atleti second match' : atleti_second_match,
        'competitor second match' : competitor_second_match,
        
     }

    df = pd.DataFrame(dict)
    return df

In [28]:
#Uefa champions league play offs

In [29]:
# Round 16 ucl 

In [30]:
simone_R16_ucl = extract_atleti_stats_from_UEFA_Play_off_rounds('text data ucl/simone_in_R16_in_ucl.txt')

In [31]:
simone_R16_ucl

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2013/2014,1,0,4,1
1,2014/2015,0,1,1,0
2,2015/2016,0,0,0,0
3,2016/2017,4,2,0,0
4,2018/2019,2,0,0,3
5,2019/2020,1,0,3,2
6,2020/2021,0,1,0,2
7,2021/2022,1,1,1,0
8,2023/2024,0,1,2,1


In [32]:
upload_data(simone_R16_ucl, 'processed data/simone_R16_ucl.csv')

In [33]:
#Round 8 ucl 

In [34]:
simone_R8_ucl = extract_atleti_stats_from_UEFA_Play_off_rounds('text data ucl/simone_in_R8_ucl.txt')

In [35]:
simone_R8_ucl

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2013/2014,1,1,1,0
1,2014/2015,0,0,0,1
2,2015/2016,1,2,2,0
3,2016/2017,1,0,1,1
4,2018/2019,1,2,0,0
5,2022/2022,0,1,0,0
6,2022/2023,2,1,2,4


In [36]:
upload_data(simone_R8_ucl, 'processed data/simone_R8_ucl.csv')

In [37]:
#Semi final ucl 

In [38]:
simone_semi_final_ucl = extract_atleti_stats_from_UEFA_Play_off_rounds('text data ucl/simone_in_semi_final_ucl.txt')

In [39]:
simone_semi_final_ucl

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2013/2014,0,0,3,1
1,2015/2016,1,0,1,2
2,2016/2017,0,3,2,1


In [40]:
upload_data(simone_semi_final_ucl, 'processed data/simone_semi_final_ucl.csv')

In [41]:
#Final ucl

In [42]:
simone_finals_ucl = extract_atleti_stats_from_UEFA_Play_off_rounds('text data ucl/simone_in_final_ucl.txt')

In [43]:
simone_finals_ucl

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2013/2014,1,4,0,0
1,2015/2016,1,1,0,0


In [44]:
upload_data(simone_finals_ucl, 'processed data/simone_finals_ucl.csv')

## Uefa Europe League 2011-2012 Play off matches

In [45]:
round_32_simone_2012 = extract_atleti_stats_from_UEFA_Play_off_rounds('text data europe league/2011-2012/round_32_ul.txt')

In [46]:
round_32_simone_2012 

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2011/2012,3,1,1,0


In [47]:
round_16_simone_2012 = extract_atleti_stats_from_UEFA_Play_off_rounds('text data europe league/2011-2012/round_16_ul.txt')

In [48]:
round_16_simone_2012

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2011/2012,3,1,3,0


In [49]:
round_8_simone_2012  = extract_atleti_stats_from_UEFA_Play_off_rounds('text data europe league/2011-2012/round_8_ul.txt')

In [50]:
round_8_simone_2012

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2011/2012,2,1,2,1


In [51]:
semi_final_2012 = extract_atleti_stats_from_UEFA_Play_off_rounds('text data europe league/2011-2012/semi_final.txt')

In [52]:
semi_final_2012

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2011/2012,4,2,1,0


In [53]:
simone_final_2012 = extract_atleti_stats_from_UEFA_Play_off_rounds('text data europe league/2011-2012/final.txt')

In [54]:
simone_final_2012

,season,atleti first match,competitor first match,atleti second match,competitor second match
0,2011/2012,3,0,0,0


In [55]:
stage = ['R-32', 'R16', 'R8', 'semi_finals', 'finals']

europe_league_2012 = pd.concat([round_32_simone_2012, round_16_simone_2012,
                      round_8_simone_2012, semi_final_2012, simone_final_2012], ignore_index=True)
europe_league_2012['stage'] = pd.Series(stage)

In [56]:
europe_league_2012

,season,atleti first match,competitor first match,atleti second match,competitor second match,stage
0,2011/2012,3,1,1,0,R-32
1,2011/2012,3,1,3,0,R16
2,2011/2012,2,1,2,1,R8
3,2011/2012,4,2,1,0,semi_finals
4,2011/2012,3,0,0,0,finals


In [57]:
upload_data(europe_league_2012, 'processed data/europe_league_2012_atleti.csv')

In [58]:
## extract_atleti_stats_from_UEFA_Play_off_rounds pipline --> Tasks Done

In [59]:
## Uefa Europe League 2017-2018 Play_off

In [60]:
def extract_atleti_goals_in_europe_league_2017_2018(file_path):
    
    #Reading text file as csv
    df = pd.read_csv(file_path, sep='\t')

    #Creating 2 new features from score to analyze attacking and defending performance to atleti
    df[['atleti_goals', 'competitor_goals']] = df['Score'].str.split('-', expand=True)

    #the need features for the report
    dataframe = df[['Round','Scorer','atleti_goals', 'competitor_goals']]
    
    #dividing the whole dataframe by rounds 
    grouped_by_round = dataframe.groupby('Round')

    #Storing goals the atleti score 
    final_round = grouped_by_round.get_group('Final')
    quarter_finals = grouped_by_round.get_group('Quarter-finals')
    round_of_16 = grouped_by_round.get_group('Round of 16')
    round_of_32 = grouped_by_round.get_group('Round of 32')
    semi_finals = grouped_by_round.get_group('Semi-finals')

    total_goal_scored_by_atleti = [
    len(round_of_32['Scorer']),
    len(round_of_16['Scorer']),
    len(quarter_finals['Scorer']),
    len(semi_finals['Scorer']),
    len(final_round['Scorer'])]

    #Storing goals the atleti conceded
    final_round_comp = grouped_by_round.get_group('Final')
    quarter_finals_comp = grouped_by_round.get_group('Quarter-finals')
    round_of_16_comp = grouped_by_round.get_group('Round of 16')
    round_of_32_comp = grouped_by_round.get_group('Round of 32')
    semi_finals_comp = grouped_by_round.get_group('Semi-finals')

    R_32_comp = round_of_32_comp['competitor_goals'].unique().astype(int)
    R_16_comp = round_of_16_comp['competitor_goals'].unique().astype(int)
    R_8_comp = quarter_finals_comp['competitor_goals'].unique().astype(int)+1
    semi_comp = semi_finals_comp['competitor_goals'].unique().astype(int)
    final_comp = final_round_comp['competitor_goals'].unique().astype(int)

    total_goal_scored_by_compatitor=[sum(R_32_comp), sum(R_16_comp), sum(R_8_comp), sum(semi_comp), sum(final_comp)]
    stage_name =['R_32', 'R_16', 'R_8', 'semi-finals', 'final']
    
    dict = { 'stage':stage_name, 
         'scored goals' : total_goal_scored_by_atleti, 
         'conceded goals' :total_goal_scored_by_compatitor }
    df = pd.DataFrame(dict)

    return df

In [61]:
#uploading atleti uefa Europe League 2017-2018 stats

In [62]:
atleti_europe_league_2018 = extract_atleti_goals_in_europe_league_2017_2018('text data europe league/2017_2018.txt')

In [63]:
atleti_europe_league_2018

,stage,scored goals,conceded goals
0,R_32,5,1
1,R_16,8,1
2,R_8,2,1
3,semi-finals,2,1
4,final,3,0


In [64]:
## highest teams scored and conceded goals in uefa champions league and europe league

In [65]:
def ranked_lists_to_df_in_uefa (file_path) :
    
    with open(file_path, 'r') as text_file:
        content = text_file.read()

    separated_content = content.split('\n')

    team_name_rank_nation = []
    total_scored_goals = []

    for i in range (0, len(separated_content), 1):
        if i==0 or i%2==0 :
            team_name_rank_nation.append(separated_content[i])
        else :
            total_scored_goals.append(separated_content[i])

    dict = {'team name, rank, nation': team_name_rank_nation,
        'total goals': total_scored_goals
    }
    df = pd.DataFrame(dict)

    return df


In [66]:
#uefa champions league 

In [67]:
top_teams_scores_goals_in_ucl_df = ranked_lists_to_df_in_uefa('text data ucl/top_teams_scored_in_ucl.txt')

In [68]:
top_teams_scores_goals_in_ucl_df

,"team name, rank, nation",total goals
0,1 Real Madrid ESP,1075
1,2 Bayern MÃ¼nchen GER,825
2,3 Barcelona ESP,687
3,4 Man Utd ENG,545
4,5 Benfica POR,479
...,...,...
95,96 Lokomotiv Moskva RUS,56
96,96 BrÃ¸ndby DEN,56
97,98 CFR Cluj ROU,54
98,98 Lille FRA,54


In [69]:
upload_data(top_teams_scores_goals_in_ucl_df, 'processed data/top_teams_scores_goals_in_ucl_df.csv')

In [70]:
top_teams_conceded_goal_in_ucl_dataframe = ranked_lists_to_df_in_uefa('text data ucl/most_goals_conceded_in_ucl.txt')

In [71]:
top_teams_conceded_goal_in_ucl_dataframe.head()

,"team name, rank, nation",total goals
0,1 Real Madrid ESP,536
1,2 Bayern MÃ¼nchen GER,386
2,3 Barcelona ESP,357
3,4 Benfica POR,345
4,5 Dynamo Kyiv UKR,320


In [72]:
upload_data(top_teams_conceded_goal_in_ucl_dataframe,'processed data/top_teams_conceded_goal_in_ucl_dataframe.csv')

In [73]:
#uefa europe league 

In [74]:
most_teams_scores_goal_in_europe_league_df = ranked_lists_to_df_in_uefa('text data europe league/top_teams_score_in_europeLeague.txt')

In [75]:
most_teams_scores_goal_in_europe_league_df.head()

,"team name, rank, nation",total goals
0,1 Sporting CP POR,320
1,2 Tottenham ENG,315
2,3 PSV NED,305
3,4 Roma ITA,302
4,5 Inter ITA,297


In [76]:
upload_data(most_teams_scores_goal_in_europe_league_df,'processed data/most_teams_scores_goal_in_europe_league_df.csv')

In [77]:
most_teams_conceded_goal_in_europe_league_df = ranked_lists_to_df_in_uefa('text data europe league/most_team_conceded_goals_in_europeLeague.txt')

In [78]:
most_teams_conceded_goal_in_europe_league_df

,"team name, rank, nation",total goals
0,1 Club Brugge BEL,236
1,2 SK Rapid AUT,229
2,3 Sporting CP POR,226
3,4 AEK Athens GRE,212
4,5 Partizan SRB,206
...,...,...
95,96 Glentoran NIR,102
96,97 VitÃ³ria SC POR,101
97,97 HonvÃ©d HUN,101
98,99 H. Tel Aviv ISR,100


In [79]:
upload_data(most_teams_conceded_goal_in_europe_league_df,'processed data/most_teams_conceded_goal_in_europe_league_df.csv')